In [1]:
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
plt.style.use('seaborn-whitegrid')
from scipy import optimize

import Model_project as mp
%load_ext autoreload
%autoreload 2

In this model project we want to solve the Solow model with humancapital. We take the model from 'Introducing Advanced Macroeconomics - Growth and Business Cycles'. 3 edition 2022, Sørensen, Peter Birch and Whitta-Jacobsen, Hans Jørgen.

The Solow model with human capital from Mankiw, Weil and Romer is an extenstion of the general Solow model, that helps explain why international investments does not flow to poorer countries as predicted by the general Solow model. The modification finds itself in the addition of human capital, which is mostly measured as years of education and experience in general, and sheds greater light on why some countries are rich and some are poor. 

# The baseline model

In [2]:
#First we define all the parameters and variables using sympy
Y, K, alpha, H, phi, A, L, y, r, w, n, g, K1, sk, sh, H1, delta, L1, A1, tau, h_tilde, h_tilde1, k_tilde, k_tilde1,\
k_tilde_star, h_tilde_star, y_star, Y_ex, K_ex, alpha_ex, H_ex, phi_ex, A_ex, L_ex, y_ex, n_ex, g_ex, K1_ex, sk_ex,\
sh_ex, H1_ex, delta_ex, L1_ex, A1_ex, tau_ex, h_tilde_ex, h_tilde1_ex, k_tilde_ex, k_tilde1_ex, k_tilde_star_ex,\
h_tilde_star_ex, y_star_ex = sm.symbols('Y_t, K_t, alpha, H_t, phi, A_t, L_t, y_t, r_t, w_t, n, g, K_{t+1}, s_K, s_H,\
H_{t+1}, delta, L_{t+1}, A_{t+1}, tau, htilde_t, htilde_t+1, ktilde_t, ktilde_t+1, ktilde^*, htilde^*, y^*, Y_t, K_t,\
alpha, H_t, phi, A_t, L_t, y_t, n, g, K_{t+1}, s_K, s_H, H_{t+1}, delta, L_{t+1}, A_{t+1}, tau, htilde_t,\
htilde_t+1, ktilde_t, ktilde_t+1, ktilde^*, htilde^*, y^*') 

In [3]:
#Here we create all the equations of the model and displays them using sympy
Y_t = sm.Eq(Y, (K**alpha*H**phi)*(A*L)**(1-alpha-phi))
r_t = sm.Eq(r, alpha*(K/(A*L))**(alpha-1)*(H/(A*L))**phi)
w_t = sm.Eq(w, (1-alpha)*(K/(A*L))**alpha*(H/(A*L))**phi*A)
K_t1 = sm.Eq(K1, sk*Y+(1-delta)*K)
H_t1 = sm.Eq(H1, sh*Y+(1-delta)*H)
L_t1 = sm.Eq(L1, (1+n)*L)
A_t1 = sm.Eq(A1, (1+g)*A)
display(Y_t, r_t, w_t, K_t1, H_t1, L_t1, A_t1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(r_t, alpha*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**(alpha - 1))

Eq(w_t, A_t*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**alpha*(1 - alpha))

Eq(K_{t+1}, K_t*(1 - delta) + Y_t*s_K)

Eq(H_{t+1}, H_t*(1 - delta) + Y_t*s_H)

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

The Solow model with human capital is given with these equations. 1) stating that output is given as a Cobb-Douglas production function. 2 and 3) stating that the real rental rate on capital and real wage is given from the marginalproduct of capital and labour respectively. 4 and 5) stating the accumulation equation of capital and human capital. 6) stating that labour grows with the exogeneously rate of n. 7) stating that technology grows with the exogeneously rate of g.

# Solving the model symbolically 

First we would like to solve the model symbolically with sympy meaning that we find the steady state values to thereafter solve for different values of steady state. We begin by deriving the transition equation and thereafter we define the Solow eqautions.

The law of motion in the Solow model with human capital can be descibed by the Transition- and Solow equations. The transition equation and Solow equations states how all of the parameters in the model move over time and is stated below.

In [4]:
k_trans = sm.Eq(k_tilde1, ((1/(1+n)*(1+g))*(sk*(k_tilde**alpha)*(h_tilde**phi)+(1-delta)*k_tilde)))
h_trans = sm.Eq(h_tilde1, ((1/(1+n)*(1+g))*(sh*(k_tilde**alpha)*(h_tilde**phi)+(1-delta)*h_tilde)))
display(k_trans, h_trans)

Eq(ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K + ktilde_t*(1 - delta))/(n + 1))

Eq(htilde_t+1, (g + 1)*(htilde_t*(1 - delta) + htilde_t**phi*ktilde_t**alpha*s_H)/(n + 1))

The Solow eqautins can be found by subtracting both sides of the equation by $\tilde{k}_t$ and $\tilde{h}_t$, respectively. The Solow eqautions therefore become.

In [5]:
k_solow = sm.Eq(k_tilde1-k_tilde, ((1/(1+n)*(1+g))*(sk*(k_tilde**alpha)*(h_tilde**phi)-(n+g+delta+n*g)*k_tilde)))
h_solow = sm.Eq(h_tilde1-h_tilde, ((1/(1+n)*(1+g))*(sh*(k_tilde**alpha)*(h_tilde**phi)-(n+g+delta+n*g)*k_tilde)))
display(k_solow, h_solow)

Eq(-ktilde_t + ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n))/(n + 1))

Eq(-htilde_t + htilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_H - ktilde_t*(delta + g*n + g + n))/(n + 1))

As we know analytically these equations can be solved to find steady state by setting $\tilde{k}_{t+1}$ and $\tilde{k}_{t}$ equal to eachother and solving for $\tilde{k}_{t}$ and the same for $\tilde{h}_{t+1}$ and $\tilde{h}_{t}$. That is the same as solving the right hand side and the left hand side in the big parenthesis to be equal to eachother. If this is done we end up with the following steady state values:

In [6]:
k_steady_state = sm.Eq(k_tilde_star, ((sk**(1-phi)*(sh**phi))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
h_steady_state = sm.Eq(h_tilde_star, ((sk**(alpha)*(sh**(1-alpha)))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
print('So physical capital- and human capital per effective worker is in steady state equal to:')
display(k_steady_state, h_steady_state)

So physical capital- and human capital per effective worker is in steady state equal to:


Eq(ktilde^*, (s_H**phi*s_K**(1 - phi)/(delta + g*n + g + n))**(1/(-alpha - phi + 1)))

Eq(htilde^*, (s_H**(1 - alpha)*s_K**alpha/(delta + g*n + g + n))**(1/(-alpha - phi + 1)))

We can now find ${y^*}$ out of our definition of $\tilde{y^{*}}$ = $\frac{y^{*}}{A_{{t}}}$  <=> ${y^*}$ = ${A}_{t}*\tilde{k^\alpha}_{t}*\tilde{h^\phi}_{t}$

In [7]:
sk_div = ((sk/(n+g+delta+n*g))**(alpha/(1-alpha-phi))) #For more cleaner code

y_steady_state = sm.Eq(y_star, (sk_div*(sh/(n+g+delta+n*g))**(phi/(1-alpha-phi)))*A)
print('GDP per worker is in steady state equal to')
display(y_steady_state)

GDP per worker is in steady state equal to


Eq(y^*, A_t*(s_H/(delta + g*n + g + n))**(phi/(-alpha - phi + 1))*(s_K/(delta + g*n + g + n))**(alpha/(-alpha - phi + 1)))

Now that we have all of our steady state values we can begin to calculate them by sympy for given parameter values.
From Weil, Mankiw and Romer (taking from the book) we get the following 'realistic' parameter values for all of our exogenous variables.

In [8]:
delta = 0.06
n = 0
g = 0.015
alpha = (1/3)
phi = (1/3)
sk = 0.2
sh = 0.15
tau = 0.15
A = 1 #Most estimates set A to be around 1, so therefore the value that it has

Now all there is left to do is to restate the equations, but now with the parameter values as given from above.

In [9]:
sk_div2 = ((sk/(n+g+delta+n*g))**(alpha/(1-alpha-phi))) #For more cleaner code

k_steady_state_value = sm.Eq(k_tilde_star, (((sk**(1-phi))*(sh**phi))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
h_steady_state_value = sm.Eq(h_tilde_star, (((sk**alpha)*(sh**(1-alpha)))/(n+g+delta+n*g))**(1/(1-alpha-phi)))
y_steady_state_value = sm.Eq(y_star, (((sk_div2*((sh/(n+g+delta+n*g))**(phi/(1-alpha-phi))))*A)))
print('With the given parameter values. Physical capital per effective worker is in steady state:')
display(k_steady_state_value)
print('With the given parameter values. Human capital per effective worker is in steady state:')
display(h_steady_state_value)
print('With the given parameter values. GDP per worker is in steady state:')
display(y_steady_state_value)

With the given parameter values. Physical capital per effective worker is in steady state:


Eq(ktilde^*, 14.2222222222222)

With the given parameter values. Human capital per effective worker is in steady state:


Eq(htilde^*, 10.6666666666667)

With the given parameter values. GDP per worker is in steady state:


Eq(y^*, 5.33333333333333)

Not so weirdly we get a high value of $\tilde{k^*}$ because of the savings rate on physical capital being higher than on human capital. we do however in the expressions see something odd. That is both human capital and physical capital depends on both the savings rate of human capital and the savings rate of physical capital. We find the explaination to that in something called self-reinforcing- and cross effects. The effects can be seen in that a higher savings rate on human capital will infest itself in a higher GDP per (effective) worker which will in itself bring on more savings to both human- and physical capital which will further root itself in higher levels of physical- and human capital and so on. This will continue for infinty, though the effect will at last become almost equal to 0. That is why we see such a high level of GDP per worker in steady state.

# Solving the model numerically

Now that we have solved the model symbolically there is one thing left to do. That is, solve it numerically and compare the two results. We will solve the model numerically by using scipy and we will optimize with an algorithm, were in we will use the bisection algorithm.

So as we said we are going to solve the model numerically by using an algorithm. But first, what is an algorithm?
An algorithm can be thought off as a recipe to solve a problem, just like when you cook, you use a recipe. An algorithm therefore contains different steps and the one we are going to use is no different.
Down below we describe the steps of the alogrithm that we are going to use:

Before we even start we have to know what we are going to solve.
So the problem that we are going to calculate is: Find the root of the steady state function with the given parameter values.

In mathematical terms it is going to look like this:

In [10]:
sk_sh = ((sk_ex**(1-phi_ex))*(sh_ex**phi_ex))

k_ss_num = sm.Eq(k_tilde_star-(((sk_ex**(1-phi_ex))*(sh_ex**phi_ex))/(n_ex+g_ex+delta_ex+n_ex*g_ex))**(1/(1-alpha_ex-phi_ex)),)
k_ss_num

/Users/lucasbillot/opt/anaconda3/lib/python3.9/site-packages/sympy/core/relational.py:495: SymPyDeprecationWarning: 

Eq(expr) with rhs default to 0 has been deprecated since SymPy 1.5.
Use Eq(expr, 0) instead. See
https://github.com/sympy/sympy/issues/16587 for more info.

  SymPyDeprecationWarning(


Eq(ktilde^* - (s_H**phi*s_K**(1 - phi)/(delta + g*n + g + n))**(1/(-alpha - phi + 1)), 0)

And obvivously the same for $\tilde{h}_{t}$ and ${y^*}$

So the first step in the algorithm is to set ${a}_{0}$ and ${b}_{0}$ equal to a and b respectively. Were $f(a)$ and $f(b)$ has opposite sign, $f(a_0)f(b_0)<0$. In other words, we take two point with function values lower and higher than a threshold value, with one of the values being negative and the other being positive.  

The next step is to Compute $f(m_0)$ where $m_0 = (a_0 + b_0)/2$ is the midpoint. Again in other words this means that we calculate the midpoint between the two function values 

The third step is thereafter to  Determine the next sub-interval $[a_1,b_1]$:
  * If $f(a_0)f(m_0) < 0$ (different signs) then $a_1 = a_0$ and $b_1 = m_0$ (i.e. focus on the range $[a_0,m_0]$).
  * If $f(m_0)f(b_0) < 0$ (different signs) then $a_1 = m_0$ and $b_1 = b_0$ (i.e. focus on the range $[m_0,b_0]$). 
This means that when we have calculated the midpoint between the two function values, we move there. The first * says that if the midpoint is below the threshold value look in a further up interval. The second * says exactly the opposite, so if the midpoint is above the threshold value look in an interval below.

The last step is to Repeat step 2 and step 3 until $f(m_n) < \epsilon$. 

To really understand this try to follow along this analogy. If you imagine back in the days that you wanted to call your aunt, called Mette, but didnt know her number, so you had to look it up in a phonebook. So you start by turning to almost the last page and right after almost the first page in the phonebook, one with the names starting with a c and one with the names starting with a, v. You realize that you are far from the page with the names starting with and m, so you go to the page right in the middle of the letter v and s. This brings you to the page with the names starting with a l. You realize that this is below m in the alphabet so you turn to the page in the middle of the interval of the two letters l and z. This keeps going until you finally end up at the page with the names starting with a m. 

See also this https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html

In [11]:
#a. first we create the objective function via numpys lambda implementation
k_ss_func = lambda k: k-(((sk**(1-phi)*(sh**phi))/(n+g+delta+n*g))**(1/(1-alpha-phi)))

In [12]:
def bisect(a, b, alpha=(1/3), phi=(1/3), g=0.015, sk=0.2, sh=0.15, n=0, delta=0.06):
    """ bisection
    
     Solve equation f(x) = 0 for a <= x <= b.
    
    Args:
    
        a      (float)          : one end of the bracketing interval [a,b]
        b      (float)          : the other end of the bracketing interval [a,b]
        alpha  (float)          : income share of capital and labour
        phi    (float)          : income share of human capital and labour
        g      (float)          : growth rate of technology
        sk     (float)          : savings rate of physical capital
        sh     (float)          : savings rate of human capital
        n      (float)          : growth rate of labour
        delta  (float)          : depreciation rate on human- and physcial capital
    
    Returns:
    
        steady state value for k(tilde)^*
        
    """
        
    alpha = alpha
    sk = sk
    sh = sh
    n = n
    g = g
    phi = phi
    delta = delta
    return optimize.bisect(k_ss_func, a, b, full_output=True)

print('Steady state of physical capital per effective worker, found numerically, is:')
print(bisect(12,16))

Steady state of physical capital per effective worker, found numerically, is:
(14.222222222220807,       converged: True
           flag: 'converged'
 function_calls: 43
     iterations: 41
           root: 14.222222222220807)


We can know calculate the steady state values of human capital per effective worker and GDP per woker, numerically, exactly the same way as we just did. If we do that then we will end up with:

In [23]:
print('Steady state of human capital per effective worker, found numerically, is:')
print(mp.bisect2(8,12))
print('Steady state of GDP per worker, found numerically, is:')
print(mp.bisect3(4,8))

Steady state of human capital per effective worker, found numerically, is:
(10.66666666666606,       converged: True
           flag: 'converged'
 function_calls: 43
     iterations: 41
           root: 10.66666666666606)
Steady state of GDP per worker, found numerically, is:
(5.33333333333394,       converged: True
           flag: 'converged'
 function_calls: 43
     iterations: 41
           root: 5.33333333333394)


As we can see we get the exact same answers whether or not we solve the model symbolically via sympy or numerically via the bisection algorithm. We have stated the symbolical answers again down below, just for the readers sake.

In [24]:
display(k_steady_state_value)
display(h_steady_state_value)
display(y_steady_state_value)

Eq(ktilde^*, 14.2222222222222)

Eq(htilde^*, 10.6666666666667)

Eq(y^*, 5.33333333333333)

# Interactive phase diagram

# Extended model with taxation/government 

We extend the model by including taxation on wealth and thereby adding an 'active' government. We add a taxation on physical capital (wealth) and add a government who uses all of the tax provenue to invest in human capital. Meaning that human capital will now only accumulate on the basis of physical capital, the government and the tax rate. See this PDF document: https://absalon.ku.dk/courses/55956/files/5668197?module_item_id=1559108

The extended Solow model with human capital and taxation is given by:

In [25]:
Y_ext = sm.Eq(Y_ex, (H_ex**phi_ex)*(K_ex**alpha)*((A_ex*L_ex)**(1-alpha_ex-phi_ex)))
L_ext = sm.Eq(L1_ex, L_ex*(1+n))
A_ext = sm.Eq(A1_ex, A_ex*(1+g))
K_t_1 = sm.Eq(K1_ex, sk_ex+Y_ex+(1-tau_ex-delta_ex)*K_ex)
H_t_1 = sm.Eq(H1_ex, tau_ex*K_ex+(1-tau_ex)*H_ex)
display(Y_t, L_t1, A_t1, K_t_1, H_t_1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

Eq(K_{t+1}, K_t*(-delta - tau + 1) + Y_t + s_K)

Eq(H_{t+1}, H_t*(1 - tau) + K_t*tau)

The extended Solow model is given by these equations. 1) Stating that output is a generel Cobb-Douglas production function. 2 and 3) Stating that labour and technology grows with the exogenously rates of n and g respectively. 4) Stating that capital accumulates by the total amount of savings and last years capital, now including a tax rate. 5) Stating that human capital know accumulates only by the amount of tax provenue from physical capital and last periods amount of human capital.

# Solving the model symbolically

The solving of this model is pretty similar to the baseline model, but nonetheless our goal is still to find the steady state values for $\tilde{k}_t$, $\tilde{h}_t$ and ${y^*}$ and see if it makes any difference to include a tax on wealth income.

The law of motion in this model is a little bit different since the transition equation and thereby the Solow equations have changed a little bit. The transition equations are stated below:

In [26]:
k_and_h_tilde = (k_tilde_ex**alpha_ex)*(h_tilde_ex**phi_ex) #For more cleaner code

k_ex_trans = sm.Eq(k_tilde1_ex, ((1/(1+n_ex)*(1+g_ex))*(sk_ex*k_and_h_tilde+(1-delta_ex-tau_ex)*k_tilde_ex)))
h_ex_trans = sm.Eq(h_tilde1_ex, ((1/(1+n_ex)*(1+g_ex))*(tau_ex*k_tilde_ex+(1-delta_ex)*h_tilde_ex)))
print('The transition equations for the extended Solow model is as follows:')
display(k_ex_trans, h_ex_trans)

The transition equations for the extended Solow model is as follows:


Eq(ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K + ktilde_t*(-delta - tau + 1))/(n + 1))

Eq(htilde_t+1, (g + 1)*(htilde_t*(1 - delta) + ktilde_t*tau)/(n + 1))

The Solow equations can now again be found by subtracting both sides of the equation with $\tilde{k}_{t}$ and $\tilde{h}_{t}$, respectively. The Solow eqautions of the extended Solow model is now given by:

In [27]:
down_paren = (n_ex+g_ex+n_ex*g_ex+delta_ex) #For more cleaner code

k_ex_solow = sm.Eq(k_tilde1_ex-k_tilde_ex, ((1/(1+n_ex)*(1+g_ex))*(sk_ex*k_and_h_tilde-(down_paren+tau_ex)*k_tilde_ex)))
h_ex_solow = sm.Eq(h_tilde1_ex-h_tilde, ((1/(1+n_ex)*(1+g_ex))*(tau_ex*k_tilde_ex-(down_paren)*h_tilde_ex)))
print('The Solow equations for the extended Solow model is:')
display(k_ex_solow, h_ex_solow)

The Solow equations for the extended Solow model is:


Eq(-ktilde_t + ktilde_t+1, (g + 1)*(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n + tau))/(n + 1))

Eq(-htilde_t + htilde_t+1, (g + 1)*(-htilde_t*(delta + g*n + g + n) + ktilde_t*tau)/(n + 1))

Now that we have our Solow equations we can again derive our steady state values for $\tilde{k}_{t}$ and $\tilde{h}_{t}$ by setting the right hand side and the left hand side in the big parenthesis eqaul to eachother. If this is done we end up with the following steady state values:

In [28]:
power11 = (1/(1-alpha_ex-phi_ex)) #For more cleaner code

k_ex_ss = sm.Eq(k_tilde_star_ex,((sk_ex/(down_paren+tau_ex))*((tau_ex**phi_ex)/(down_paren**phi_ex)))**power11)
h_ex_ss = sm.Eq(h_tilde_star_ex,(((sk_ex/(down_paren+tau_ex))*((tau_ex**(1-alpha_ex))/((down_paren+tau_ex)**(1-alpha_ex))))**power11))
print('The steady state of capital per effective worker in the extended Solow model is given by:')
display(k_ex_ss)
print('The steady state of human capita per effective worker in the extended Solow model is given by:')
display(h_ex_ss)

The steady state of capital per effective worker in the extended Solow model is given by:


Eq(ktilde^*, (s_K*tau**phi/((delta + g*n + g + n)**phi*(delta + g*n + g + n + tau)))**(1/(-alpha - phi + 1)))

The steady state of human capita per effective worker in the extended Solow model is given by:


Eq(htilde^*, (s_K*tau**(1 - alpha)*(delta + g*n + g + n + tau)**(alpha - 1)/(delta + g*n + g + n + tau))**(1/(-alpha - phi + 1)))

The steady state for GDP per worker can again be found on the definition of GDP per worker:

In [29]:
power1 = (phi_ex/(1-alpha_ex-phi_ex)) 
power2 = (alpha_ex+phi_ex)/(1-alpha_ex-phi_ex) #For more cleaner code. Both of these lines

y_ex_ss = sm.Eq(y_star_ex,A_ex*((tau_ex/down_paren)**power1)*((sk_ex/(down_paren+tau_ex))**power2))
print('The steady state of GDP per worker is in the extended Solow model given by:')
display(y_ex_ss)

The steady state of GDP per worker is in the extended Solow model given by:


Eq(y^*, A_t*(s_K/(delta + g*n + g + n + tau))**((alpha + phi)/(-alpha - phi + 1))*(tau/(delta + g*n + g + n))**(phi/(-alpha - phi + 1)))

Now that we have our expression for the steady state values of importance we would like to solve them symbolically. We do this by inserting the given parameter values into the expressions. Therefore we make 3 new equations that do exactly this.

In [30]:
down_paren2 = (n+g+delta+n*g)
power111 = (1/(1-alpha-phi))
power22 = ((alpha+phi)/(1-alpha-phi)) #Again both of these lines are just because of cleaner code

k_ex_ss_value = sm.Eq(k_tilde_star_ex,(((sk/(n+g+delta+tau+n*g))*((tau**phi)/((n+g+delta+n*g)**phi)))**(1/(1-alpha-phi))))
h_ex_ss_value = sm.Eq(h_tilde_star_ex,(((sk/(down_paren2+tau))*((tau**(1-alpha))/((down_paren2+tau)**(1-alpha))))**power111))
y_ex_ss_value = sm.Eq(y_star_ex,A*(((tau/(down_paren2))**(phi/(1-alpha-phi)))*((sk/(down_paren2+tau))**power22)))
print('With the given parameter values. Physical capital per effective worker is in steady state:')
display(k_ex_ss_value)
print('With the given parameter values. Human capital per effective worker is in steady state:')
display(h_ex_ss_value)
print('With the given parameter values. GDP per worker is in steady state:')
display(y_ex_ss_value)

With the given parameter values. Physical capital per effective worker is in steady state:


Eq(ktilde^*, 1.40466392318244)

With the given parameter values. Human capital per effective worker is in steady state:


Eq(htilde^*, 0.312147538484987)

With the given parameter values. GDP per worker is in steady state:


Eq(y^*, 1.58024691358025)

ADD some intuition and a conclusion/closing remarks!!!

# Solving the model numerically

In [32]:
print('Steady state of physical capital per effective worker, found numerically, is:')
print(mp.bisect4(1,3))

print('Steady state of human capital per effective worker, found numerically, is:')
print(mp.bisect5(0,1))

print('Steady state of GDP per worker, found numerically, is:')
print(mp.bisect6(1,2))

Steady state of physical capital per effective worker, found numerically, is:
(1.4046639231837617,       converged: True
           flag: 'converged'
 function_calls: 42
     iterations: 40
           root: 1.4046639231837617)
Steady state of human capital per effective worker, found numerically, is:
(0.31214753848507826,       converged: True
           flag: 'converged'
 function_calls: 41
     iterations: 39
           root: 0.31214753848507826)
Steady state of GDP per worker, found numerically, is:
(1.5802469135796855,       converged: True
           flag: 'converged'
 function_calls: 41
     iterations: 39
           root: 1.5802469135796855)


Higher tau equals lower values for all three, so they are negative dependent on tau.

# Interactive phase diagram

## Graph showing the different ${y^*}$ values, for the two models